# Tic Tac Toe Game

In [ ]:
import cv2
import mediapipe as mp
import math

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize Video Capture.
cap = cv2.VideoCapture(0)

# Function to check if there is a winner.
def check_winner(board):
    # Check rows
    for row in board:
        if row[0] == row[1] == row[2] != ' ':
            return row[0]
    # Check columns
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != ' ':
            return board[0][col]
    # Check diagonals
    if board[0][0] == board[1][1] == board[2][2] != ' ':
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != ' ':
        return board[0][2]
    return None

# Function to check if the board is full.
def check_draw(board):
    for row in board:
        if ' ' in row:
            return False  # If any cell is empty, the game is not a draw
    return True  # If no empty cells and no winner, it's a draw

# Function to draw the Tic Tac Toe board on the image.
def draw_board(image, board):
    # Draw vertical lines
    cv2.line(image, (150, 0), (150, 450), (255, 255, 255), 2)
    cv2.line(image, (300, 0), (300, 450), (255, 255, 255), 2)
    # Draw horizontal lines
    cv2.line(image, (0, 150), (450, 150), (255, 255, 255), 2)
    cv2.line(image, (0, 300), (450, 300), (255, 255, 255), 2)
    # Draw X's and O's
    for i in range(3):
        for j in range(3):
            if board[i][j] == 'X':
                cv2.putText(image, 'X', (j * 150 + 50, i * 150 + 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 5)
            elif board[i][j] == 'O':
                cv2.putText(image, 'O', (j * 150 + 50, i * 150 + 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5)
xcount = 0
ocount = 0
# Game loop
while True:
    replay_game = True  # Flag to indicate whether the game should be replayed

    while replay_game:
        # Initialize game variables.
        board = [[' ']*3 for _ in range(3)]  # 3x3 Tic Tac Toe board
        current_player = 'X'  # Player 'X' starts

        while cap.isOpened():
            success, image = cap.read()
            image = cv2.flip(image, 1)
            if not success:
                print("Ignoring empty camera frame.")
                continue

            # Convert the image from BGR to RGB.
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to pass by reference.
            image_rgb.flags.writeable = False

            # Process the image with MediaPipe Hands.
            results = hands.process(image_rgb)

            # Check if hand(s) detected.
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Initialize finger coordinates.
                    index_finger_tip = None
                    thumb_tip = None

                    # Get the index of the index finger tip and thumb tip landmarks.
                    index_finger_tip_index = mp_hands.HandLandmark.INDEX_FINGER_TIP.value
                    thumb_tip_index = mp_hands.HandLandmark.THUMB_TIP.value

                    if hand_landmarks.landmark:
                        # Get the coordinates of the index finger tip.
                        index_finger_tip = hand_landmarks.landmark[index_finger_tip_index]
                        index_finger_tip_x = int(index_finger_tip.x * image.shape[1])
                        index_finger_tip_y = int(index_finger_tip.y * image.shape[0])

                        # Get the coordinates of the thumb tip.
                        thumb_tip = hand_landmarks.landmark[thumb_tip_index]
                        thumb_tip_x = int(thumb_tip.x * image.shape[1])
                        thumb_tip_y = int(thumb_tip.y * image.shape[0])

                        # Calculate the distance between the index finger tip and thumb tip.
                        distance = math.sqrt((index_finger_tip_x - thumb_tip_x)**2 + (index_finger_tip_y - thumb_tip_y)**2)

                        # Determine the cell corresponding to the finger position if the fingers are close together.
                        if distance < 25:
                            cell_x = index_finger_tip_x // 150
                            cell_y = index_finger_tip_y // 150
                            # If the cell is empty and the game is not over, make a move.
                            if 0 <= cell_x < 3 and 0 <= cell_y < 3 and board[cell_y][cell_x] == ' ' and not check_winner(board) and not check_draw(board):
                                board[cell_y][cell_x] = current_player
                                current_player = 'O' if current_player == 'X' else 'X'

            # Draw the Tic Tac Toe board on the image.
            draw_board(image, board)
            winner = check_winner(board)
            if winner == 'X':
                xcount += 1
            elif winner == 'O': 
                ocount += 1
            cv2.putText(image, f"X:{xcount}", (475, int((image.shape[0] - 300) / 2)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 4)
            cv2.putText(image, f"O:{ocount}", (475, int((image.shape[0] - 300) / 2) + 300), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,255), 4)
            
            if winner:
                cv2.putText(image, f"Winnear is : {winner}", (75, int((image.shape[0] - 300) / 2) + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0,0), 4)
                cv2.putText(image, "press r to reply", (75, int((image.shape[0] - 300) / 2) + 90 + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0,0), 4)
                cv2.putText(image, "press q to quit", (75, int((image.shape[0] - 300) / 2) + 180 + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0,0), 4)
                
            elif check_draw(board):
                cv2.putText(image, "It's a draw!", (75, int((image.shape[0] - 300) / 2) + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0,0), 4)
                cv2.putText(image, "press r to reply", (75, int((image.shape[0] - 300) / 2) + 90 + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0,0), 4)
                cv2.putText(image, "press q to quit", (75, int((image.shape[0] - 300) / 2) + 180 + 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0,0), 4)
                
            # Display the image.
            cv2.imshow('Tic Tac Toe', image)

            # Check for a winner or draw.
            if winner or check_draw(board):
                break

            # Break the loop if 'q' is pressed.
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break
            
           
        key = cv2.waitKey(0)
        if key == ord('r'):
            break
        elif key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            exit()

# Release the VideoCapture and close all windows.
cap.release()
cv2.destroyAllWindows()
